In [297]:
import pandas as pd
import numpy  as np

import data file

In [298]:
df = pd.read_csv('data (1).csv')
df.describe()

,Label
count,2.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


parse data from string to numpy array

In [299]:
def parseData(series):
    final_series = []
    for idx, ser in enumerate(series):
        temp = []
        for x in "[],":
            ser = ser.replace(x, "")
        new_ser  = np.fromstring(ser, dtype=float, sep=" ")
        for i in range(0, len(new_ser), 3):
            chunk = [new_ser[i], new_ser[i + 1], new_ser[i + 2]]
            temp.append(chunk)
        final_series.append(temp)
    return np.array(final_series)

In [300]:
right_gaze = df["Right Gaze"]
left_gaze  = df["Left Gaze"]
right_head = df["Right HeadPose"]
left_head  = df["Left HeadPose"]

right_gaze, left_gaze, right_head, left_head = parseData(right_gaze), parseData(left_gaze), parseData(right_head), parseData(left_head)

C:\Users\lViet\AppData\Local\Temp\ipykernel_10972\1516899369.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(final_series)


get the longest sequence of vector in dataset

In [301]:
max_length = 0
for (lgaze, rgaze, lhead, rhead) in zip(left_gaze, right_gaze, left_head, right_head):
    if len(lgaze) > max_length:
        max_length = len(lgaze)
    if len(rgaze) > max_length:
        max_length = len(rgaze) 
    if len(rhead) > max_length:
        max_length = len(rhead)
    if len(lhead) > max_length:
        max_length = len(lhead)
        
max_length

591

pad the dataset to the longest sequence size

In [302]:
import math

def padData(series, length):
    for idx, ser in enumerate(series):
        times = math.floor(length / len(ser))
        add = length % len(ser)
        
        temp = ser[::-1]
        for _ in range(1, times):
            series[idx] = np.append(series[idx], temp, axis=0)
            temp = temp[::-1]
        if add != 0:
            series[idx] = np.append(series[idx], temp[0:add], axis=0)
    return series

initialize new data frame

In [303]:
left_head  = padData(left_head,  max_length)
right_head = padData(right_head, max_length)
left_gaze  = padData(left_gaze,  max_length)
right_gaze = padData(right_gaze, max_length)

final_df = pd.DataFrame({
    'Left Gaze': left_gaze,
    'Right Gaze': right_gaze,
    'Left Head': left_head,
    'Right Head': right_head,
})

In [304]:
final_df

,Left Gaze,Right Gaze,Left Head,Right Head
0,"[[23.477, 49.405, 54.699], [9.526, 57.898, 58....","[[20.315, 43.502, 48.012], [1.177, 59.03, 59.0...","[[21.109, 32.571, 38.814], [4.383, 46.385, 46....","[[29.421, 26.253, 39.431], [9.98, 42.063, 43.2..."
1,"[[36.698, 41.763, 55.596], [37.742, 33.712, 50...","[[13.362, 41.782, 43.866], [3.857, 41.621, 41....","[[35.924, 49.926, 61.507], [38.748, 37.83, 54....","[[13.117, 44.598, 46.487], [6.584, 34.265, 34...."


Convolution test

In [305]:
import tensorflow as tf

y = tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(max_length, 4))(final_df)


AttributeError: 'tuple' object has no attribute 'rank'